In [1]:
%matplotlib inline


Sequence Models and Long-Short Term Memory Networks
===================================================




In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd


torch.manual_seed(1)

In [3]:
lstm = nn.LSTM(3, 3)  # 输入维度是3, 输出维度也是3
inputs = [autograd.Variable(torch.randn((1, 3)))
          for _ in range(5)]  # 构造一个长度为5的序列
print('inputs:\n',inputs)
# 初始化隐藏状态
# 实际上是初始化了隐藏元和记忆元（其实就是用来存储当前lstm单元的输出的）
# 初始化的隐藏元和记忆元,通常它们是维度是一样的
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn(1, 1, 3)))# 1个LSTM层，1个batch,隐藏元维度3
print('hidden:\n',hidden)
for i in inputs:
    # 将序列的元素逐个输入到LSTM
    # 经过每步操作,hidden 的值包含了隐藏状态的信息
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    # print(out)

inputs:
 [tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])]
hidden:
 (tensor([[[ 0.5438, -0.4057,  1.1341]]]), tensor([[[-1.1115,  0.3501, -0.7703]]]))


用于解释LSTM的输入：$$初始化的隐藏单元C_{t-1}+记忆单元S_{t-1}+当前序列X_t$$
![Image of Yaktocat](https://img-blog.csdn.net/20180518110325328?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lveW9mdTAwNw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

In [4]:
print(torch.cat(inputs),len(inputs))

tensor([[-0.5525,  0.6355, -0.3968],
        [-0.6571, -1.6428,  0.9803],
        [-0.0421, -0.8206,  0.3133],
        [-1.1352,  0.3773, -0.2824],
        [-2.5667, -1.4303,  0.5009]]) 5


In [5]:
# 另外, 我们还可以一次对整个序列进行训练. LSTM 返回的第一个值表示所有时刻的隐状态值,
# 第二个值表示最近的隐状态值 (因此下面的 "out"的最后一个值和 "hidden" 的值是一样的).
# 之所以这样设计, 是为了通过 "out" 的值来获取所有的隐状态值, 而用 "hidden" 的值来
# 进行序列的反向传播运算, 具体方式就是将它作为参数传入后面的 LSTM 网络.

# 增加额外的第二个维度
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
# 清空输出隐状态
hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(torch.randn((1, 1, 3))))  
out, hidden = lstm(inputs, hidden)
print('out:\n',out)
print('hidden:\n',hidden)

out:
 tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<CatBackward>)
hidden:
 (tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<ViewBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<ViewBackward>))


In [6]:
# print(len(inputs))

In [7]:

# from tensorboardX import SummaryWriter
# dummy_input = inputs[0].view(1,1,-1)[0]
# print(dummy_input)
# with SummaryWriter(comment='VGG_self') as w:
#     w.add_graph(lstm, (dummy_input,hidden,))

## 用 LSTM 来进行词性标注 Example: An LSTM for Part-of-Speech Tagging

### Prepare data:

In [8]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

# 生成单词向索引的词典
# word_to_ix = {word: i for i, word in enumerate(vocab)}
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# 实际中通常使用更大的维度如32维, 64维.
# 这里我们使用小的维度, 为了方便查看训练过程中权重的变化.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'that': 7, 'dog': 1, 'book': 8, 'Everybody': 5, 'ate': 2, 'The': 0, 'read': 6, 'apple': 4, 'the': 3}


In [9]:
print(len(word_to_ix),len(tag_to_ix))

9 3


### Create the model:

In [10]:
class LSTMTagger(nn.Module):
    # model = LSTMTagger(embedding_dim = 6, hidden_dim = 6, vocab_size = 9, tagset_size = 3)
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # 为单词编码
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        #  LSTM 以 word_embeddings 作为输入, 输出维度为 hidden_dim 的隐状态值
        self.lstm = nn.LSTM(embedding_dim, hidden_dim) # 输入维度=embedding_dim,输出维度=hidden_dim

        # 线性层将隐状态空间映射到标注空间
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # 开始时刻, 没有隐状态
        # 关于维度设置的详情,请参考 Pytorch 文档
        # 各个维度的含义是 (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
        #将整个句子进行编码
        embeds = self.word_embeddings(sentence)#sentence：句子对应的下标索引
        
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        
        # print(lstm_out.size(),'\n',lstm_out.view(len(sentence), -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        # print(tag_space)
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [11]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
print(model)

LSTMTagger(
  (word_embeddings): Embedding(9, 6)
  (lstm): LSTM(6, 6)
  (hidden2tag): Linear(in_features=6, out_features=3, bias=True)
)


In [12]:
# nn.LSTM?
print(training_data)
inputs = prepare_sequence(training_data[0][0], word_to_ix)
print(inputs)

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']), (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]
tensor([0, 1, 2, 3, 4])


### Train the model:



In [13]:
# 查看下训练前得分的值
# 注意: 输出的 i,j 元素的值表示单词 i 的 j 标签的得分
inputs = prepare_sequence(training_data[0][0], word_to_ix)#输出句子中单词对应的下标索引tensor([0, 1, 2, 3, 4])
tag_scores = model(inputs)
print(tag_scores)

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]], grad_fn=<LogSoftmaxBackward>)


In [14]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(300):  # 再次说明下, 实际情况下你不会训练300个周期, 此例中我们只是构造了一些假数据
    for sentence, tags in training_data:
        # Step 1\. 请记住 Pytorch 会累加梯度
        # 每次训练前需要清空梯度值
        model.zero_grad()

        # 此外还需要清空 LSTM 的隐状态
        # 将其从上个实例的历史中分离出来
        model.hidden = model.init_hidden()

        # Step 2\. 准备网络输入, 将其变为词索引的 Variables 类型数据
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3\. 前向传播
        tag_scores = model(sentence_in)

        # Step 4\. 计算损失和梯度值, 通过调用 optimizer.step() 来更新梯度
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# 查看训练后得分的值
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
# 句子是 "the dog ate the apple", i,j 表示对于单词 i, 标签 j 的得分.
# 我们采用得分最高的标签作为预测的标签. 从下面的输出我们可以看到, 预测得
# 到的结果是0 1 2 0 1\. 因为 索引是从0开始的, 因此第一个值0表示第一行的
# 最大值, 第二个值1表示第二行的最大值, 以此类推. 所以最后的结果是 DET
# NOUN VERB DET NOUN, 整个序列都是正确的!
print(tag_scores)

tensor([[-0.0858, -2.9355, -3.5374],
        [-5.2313, -0.0234, -4.0314],
        [-3.9098, -4.1279, -0.0368],
        [-0.0187, -4.7809, -4.5960],
        [-5.8170, -0.0183, -4.1879]], grad_fn=<LogSoftmaxBackward>)


## Exercise: Augmenting the LSTM part-of-speech tagger with character-level features
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In the example above, each word had an embedding, which served as the
inputs to our sequence model. Let's augment the word embeddings with a
representation derived from the characters of the word. We expect that
this should help significantly, since character-level information like
affixes have a large bearing on part-of-speech. For example, words with
the affix *-ly* are almost always tagged as adverbs in English.

To do this, let $c_w$ be the character-level representation of
word $w$. Let $x_w$ be the word embedding as before. Then
the input to our sequence model is the concatenation of $x_w$ and
$c_w$. So if $x_w$ has dimension 5, and $c_w$
dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the
characters of a word, and let $c_w$ be the final hidden state of
this LSTM. Hints:

* There are going to be two LSTM's in your new model.
  The original one that outputs POS tag scores, and the new one that
  outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.
  The character embeddings will be the input to the character LSTM.


